In [1]:
#import libraries
import pandas as pd
import numpy as np
import itertools
import networkx as nx
from scipy.cluster import hierarchy
import scipy.spatial.distance as ssd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
pd.options.mode.chained_assignment = None 
sns.set(font='Times New Roman')
sns.set(font_scale=1.5)
import re
import csv
import os

In [2]:
data = pd.read_csv("variant_surveillance.tsv", sep = '\t',dtype = {'Variant': 'string',
                                                                 'Is reference?': 'string',
                                                                 'Is complete?': 'string'})
data.head()

,Accession ID,Type,Clade,Pango lineage,Pangolin version,AA Substitutions,Variant,Collection date,Location,Host,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,Sequence length
0,EPI_ISL_10010236,betacoronavirus,GRA,BA.1.17,PANGO-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,NaN,0.379635,29747
1,EPI_ISL_10010227,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.009446,0.379794,29747
2,EPI_ISL_10010232,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.000134,0.379585,29747
3,EPI_ISL_3578085,betacoronavirus,GK,B.1.617.2,PLEARN-v1.8,"(N_D377Y,M_I82T,Spike_D950N,N_R203M,NSP12_G671...",VOC Delta GK (B.1.617.2+AY.*) first detected i...,2021-08-18,Europe / Turkey,Human,2021-08-23,<NA>,True,True,NaN,0.000101,0.379853,29754
4,EPI_ISL_10299251,betacoronavirus,GRA,BA.1,PLEARN-v1.8,"(NSP5_P132H,Spike_T95I,Spike_A67V,Spike_K417N,...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-17,Europe / Germany / North Rhine-Westphalia,Human,2022-02-24,<NA>,True,NaN,NaN,0.048908,0.380371,29729


In [3]:
data = data[data['Variant'].notna()]
data.head()

,Accession ID,Type,Clade,Pango lineage,Pangolin version,AA Substitutions,Variant,Collection date,Location,Host,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,Sequence length
0,EPI_ISL_10010236,betacoronavirus,GRA,BA.1.17,PANGO-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,NaN,0.379635,29747
1,EPI_ISL_10010227,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.009446,0.379794,29747
2,EPI_ISL_10010232,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.000134,0.379585,29747
3,EPI_ISL_3578085,betacoronavirus,GK,B.1.617.2,PLEARN-v1.8,"(N_D377Y,M_I82T,Spike_D950N,N_R203M,NSP12_G671...",VOC Delta GK (B.1.617.2+AY.*) first detected i...,2021-08-18,Europe / Turkey,Human,2021-08-23,<NA>,True,True,NaN,0.000101,0.379853,29754
4,EPI_ISL_10299251,betacoronavirus,GRA,BA.1,PLEARN-v1.8,"(NSP5_P132H,Spike_T95I,Spike_A67V,Spike_K417N,...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-17,Europe / Germany / North Rhine-Westphalia,Human,2022-02-24,<NA>,True,NaN,NaN,0.048908,0.380371,29729


In [4]:
#specify strings for countries 

vocs_who = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron','Epsilon','Lambda','Iota','Kappa','Mu','Eta','Zeta','Theta']

for i in vocs_who:
    
    data.loc[data['Variant'].str.contains(i), 'Variant'] = i




In [5]:
data = data[data['Variant'].isin(['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron','Epsilon','Lambda','Iota','Kappa','Mu','Eta','Zeta','Theta'])] 
data.shape

(10131719, 18)

In [6]:
%time
#specify strings for countries 

europe_list = ['Turkey', 'Germany','China' 'United Kingdom','Brazil','Israel', 'Denmark','Spain','Italy','Ireland','Cuba','Canada','Belgium','Sweden',
              'South Korea','USA','France','Japan','Austria','Netherlands','Switzerland','Ireland','Poland','Mexico','Luxembourg']

# %time
# europe_list = ["Europe", "North America","Oceania", "Asia","South America", "Africa"]
              
for i in europe_list:
    
    data.loc[data['Location'].str.contains(i), 'Location'] = i


#data.loc[data['Location'].str.contains('Turkey'), 'Location'] = 'Turkey'

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 45.3 µs


In [7]:
data.head()

,Accession ID,Type,Clade,Pango lineage,Pangolin version,AA Substitutions,Variant,Collection date,Location,Host,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,Sequence length
0,EPI_ISL_10010236,betacoronavirus,GRA,BA.1.17,PANGO-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",Omicron,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,NaN,0.379635,29747
1,EPI_ISL_10010227,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",Omicron,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.009446,0.379794,29747
2,EPI_ISL_10010232,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",Omicron,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.000134,0.379585,29747
3,EPI_ISL_3578085,betacoronavirus,GK,B.1.617.2,PLEARN-v1.8,"(N_D377Y,M_I82T,Spike_D950N,N_R203M,NSP12_G671...",Delta,2021-08-18,Turkey,Human,2021-08-23,<NA>,True,True,NaN,0.000101,0.379853,29754
4,EPI_ISL_10299251,betacoronavirus,GRA,BA.1,PLEARN-v1.8,"(NSP5_P132H,Spike_T95I,Spike_A67V,Spike_K417N,...",Omicron,2022-01-17,Germany,Human,2022-02-24,<NA>,True,NaN,NaN,0.048908,0.380371,29729


In [8]:
df = data[['Variant','Location','Collection date']]
df.head()

,Variant,Location,Collection date
0,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
1,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
2,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
3,Delta,Turkey,2021-08-18
4,Omicron,Germany,2022-01-17


In [9]:
df['Collection date'] = pd.to_datetime(df["Collection date"])
df.head()

,Variant,Location,Collection date
0,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
1,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
2,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25
3,Delta,Turkey,2021-08-18
4,Omicron,Germany,2022-01-17


In [10]:
df['Date']= df['Collection date'].dt.to_period('W')
df.head()

,Variant,Location,Collection date,Date
0,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
1,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
2,Omicron,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
3,Delta,Turkey,2021-08-18,2021-08-16/2021-08-22
4,Omicron,Germany,2022-01-17,2022-01-17/2022-01-23


In [11]:
df_weekly = df[(df['Collection date'] >= '2020-01-01') & (df['Collection date'] <='2022-05-31')].groupby(['Variant','Location', 'Date']).size().reset_index()
df_weekly.head()

,Variant,Location,Date,0
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1


In [12]:
df_weekly['Date'] = df_weekly['Date'].astype(str)
df_weekly.head()

,Variant,Location,Date,0
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1


In [13]:
df_weekly.columns = ['Variant','Location','Date','Freq']
df_weekly.head()

,Variant,Location,Date,Freq
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1


In [14]:
tmp = df_weekly['Date'].str.split(pat = '/', expand = True)
tmp.head()

,0,1
0,2021-02-15,2021-02-21
1,2021-04-12,2021-04-18
2,2021-04-26,2021-05-02
3,2021-05-03,2021-05-09
4,2021-05-10,2021-05-16


In [15]:
df_weekly2 = pd.merge(df_weekly, tmp.iloc[: , 0], left_index= True, right_index= True)
df_weekly2.head()

,Variant,Location,Date,Freq,0
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1,2021-02-15
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1,2021-04-12
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4,2021-04-26
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2,2021-05-03
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1,2021-05-10


In [16]:
df_weekly2.Variant.unique()

<StringArray>
[  'Alpha',    'Beta',   'Delta', 'Epsilon',     'Eta',   'Gamma',    'Iota',
   'Kappa',  'Lambda',      'Mu', 'Omicron',   'Theta',    'Zeta']
Length: 13, dtype: string

In [17]:
df_weekly2.columns = ['Variant', 'Location', 'WY-str', 'Freq', 'Date']
df_weekly2.head()

,Variant,Location,WY-str,Freq,Date
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1,2021-02-15
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1,2021-04-12
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4,2021-04-26
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2,2021-05-03
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1,2021-05-10


In [18]:
df_weekly2['Date'] = pd.to_datetime(df_weekly2['Date'])
df_weekly2.head()

,Variant,Location,WY-str,Freq,Date
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1,2021-02-15
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1,2021-04-12
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4,2021-04-26
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2,2021-05-03
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1,2021-05-10


In [19]:
df_weekly2.sort_values(by = 'Date')
df_weekly2.head()

,Variant,Location,WY-str,Freq,Date
0,Alpha,Africa / Algeria / Algiers,2021-02-15/2021-02-21,1,2021-02-15
1,Alpha,Africa / Algeria / Algiers,2021-04-12/2021-04-18,1,2021-04-12
2,Alpha,Africa / Algeria / Algiers,2021-04-26/2021-05-02,4,2021-04-26
3,Alpha,Africa / Algeria / Algiers,2021-05-03/2021-05-09,2,2021-05-03
4,Alpha,Africa / Algeria / Algiers,2021-05-10/2021-05-16,1,2021-05-10


In [20]:
#claculate weekly sums

df_weekly_sum = df_weekly.groupby(['Variant', 'Date', 'Location'], as_index= False)['Freq'].sum()
df_weekly_sum.head()

,Variant,Date,Location,Freq
0,Alpha,2019-12-30/2020-01-05,Africa / South Africa / Gauteng,1
1,Alpha,2019-12-30/2020-01-05,Asia / India / Kerala,5
2,Alpha,2019-12-30/2020-01-05,Asia / India / Tamil Nadu,1
3,Alpha,2019-12-30/2020-01-05,Austria,4
4,Alpha,2019-12-30/2020-01-05,Europe / Cyprus,21


In [21]:
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Epsilon']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Lambda']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Iota']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Kappa']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Mu']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Eta']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Zeta']
df_weekly_sum = df_weekly_sum[df_weekly_sum['Variant'] != 'Theta']
df_weekly_sum.head(), df_weekly_sum['Variant'].unique()

(  Variant                   Date                         Location  Freq
 0   Alpha  2019-12-30/2020-01-05  Africa / South Africa / Gauteng     1
 1   Alpha  2019-12-30/2020-01-05            Asia / India / Kerala     5
 2   Alpha  2019-12-30/2020-01-05        Asia / India / Tamil Nadu     1
 3   Alpha  2019-12-30/2020-01-05                          Austria     4
 4   Alpha  2019-12-30/2020-01-05                  Europe / Cyprus    21,
 <StringArray>
 ['Alpha', 'Beta', 'Delta', 'Gamma', 'Omicron']
 Length: 5, dtype: string)

In [22]:
#clades contains all GISAID clades, VOC contains GISAID clades containing VOCs
#change clade input to whichever you want to analyze
clades = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron','Epsilon','Lambda','Iota','Kappa','Mu','Eta','Zeta','Theta']
# VOC = ['Delta', 'Omicron']
VOC = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron']

clade_input = VOC    ####

#function to generate distance matrix based on quarter, year, and continent
def create_distance_matrix(df,quarter_continent):
    tmp = df['Variant'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    distance_matrix = pd.DataFrame(index = clade_list)
    distance_matrix[clade_list] = np.nan
    #fill in distance matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                distance_matrix.loc[i,j] = 0
            else:
                distance_matrix.loc[i,j] = (abs((df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                distance_matrix.loc[j,i] = (abs((df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    return distance_matrix

#function to generate difference matrix based on quarter, year, and continent
def create_difference_matrix(df,quarter_continent):
    tmp = df['Variant'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    difference_matrix = pd.DataFrame(index = clade_list)
    difference_matrix[clade_list] = np.nan
    #fill in difference matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                difference_matrix.loc[i,j] = 0
            else:
                difference_matrix.loc[i,j] = (abs((df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                difference_matrix.loc[j,i] = (abs((df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    #if row_vs_col is a negative value, make it negative in difference matrix
    for i,v in difference_matrix.iterrows():
        for j in difference_matrix:
            if i != j:
                if (df.loc[df['Variant'] == '{a} vs {b}'.format(a=i,b=j),quarter_continent].iloc[0]) < 0:
                    difference_matrix.loc[i,j] *= -1
    return difference_matrix

#function to calculate the log transformed ratio of two clades
def transform_ratio(clade_1, clade_2):
    return np.log10(clade_1/clade_2)

def time_frame(df, start_date, end_date):  #updated
    time_window = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m').unique().tolist()
    print(time_window)
    dataframe = df.loc[df['Collection date'].isin(time_window)]
    dataframe.iloc[:,2:] = dataframe.iloc[:,2:].cumsum()
    for i in dataframe.columns.tolist():
        if dataframe[i].isnull().all():
            dataframe = dataframe.drop(i, axis=1)
    return dataframe


# def time_frame_no_cumsum(df,time_window):
#     dataframe = df.loc[df['Collection date'].isin(time_window)]
#     return dataframe

def time_frame_no_cumsum(df,start_date, end_date): #updated
    time_window = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m').unique().tolist()
    print(time_window)
    dataframe = df.loc[df['Collection date'].isin(time_window)]
    return dataframe

#function returns slope of clade advantage plots
def slope_calc(day, pairwise_fitness):
    par = np.polyfit(day, pairwise_fitness, 1, full = True)
    slope= par[0][0]
    return slope

def clade_comparisons(df):
    clade_list = [i for i in clade_input if i in df.columns.tolist()]

    df['date_ordinal'] = pd.to_datetime(df['Date']).apply(lambda date: date.toordinal())
    df['days'] = df['date_ordinal'] - 737425
    #calculate log ratio for each clade comparison
    for i in clade_list:
        for j in clade_list:
            if i != j:
                df['{a} vs {b}'.format(a=i,b=j)] = df.apply(lambda row : 
                                                                        transform_ratio(row['{a}'.format(a=i)],
                                                                                        row['{b}'.format(b=j)]),
                                                                                        axis = 1)
    clade_list.append('date_ordinal')
    df = df.drop(clade_list, axis=1)
    return df

In [36]:
df_weekly_sum.head()

,Variant,Date,Location,Freq
0,Alpha,2019-12-30/2020-01-05,Africa / South Africa / Gauteng,1
1,Alpha,2019-12-30/2020-01-05,Asia / India / Kerala,5
2,Alpha,2019-12-30/2020-01-05,Asia / India / Tamil Nadu,1
3,Alpha,2019-12-30/2020-01-05,Austria,4
4,Alpha,2019-12-30/2020-01-05,Europe / Cyprus,21


In [37]:
df_weekly_sum = pd.read_csv('dpgr_who_countries_1.csv')
df_weekly_sum = pd.DataFrame(df_weekly_sum)
df_weekly_sum.head()

,Variant,Date,Location,Freq
0,Alpha,2019-12-30/2020-01-05,Africa / South Africa / Gauteng,1
1,Alpha,2019-12-30/2020-01-05,Asia / India / Kerala,5
2,Alpha,2019-12-30/2020-01-05,Asia / India / Tamil Nadu,1
3,Alpha,2019-12-30/2020-01-05,Austria,4
4,Alpha,2019-12-30/2020-01-05,Europe / Cyprus,21


In [38]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

clades = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron','Epsilon','Lambda','Iota','Kappa','Mu','Eta','Zeta','Theta']
VOC = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Omicron']
clade_input = VOC

def scatterplot(country, dataset, start, end, variant):
    # Filter dataset for the specified country
    df_weekly = dataset[dataset['Location'] == country]
    
    # Ensure the 'Date' column is in the correct format
    df_weekly['Date'] = pd.to_datetime(df_weekly['Date'], errors='coerce')
    
    # Check for any null dates and drop them
    df_weekly = df_weekly.dropna(subset=['Date'])
    
    # Debug print to check unique variants and dates
    print("Unique Variants:", df_weekly['Variant'].unique())
    print("Date Range after conversion and dropping nulls:", df_weekly['Date'].min(), "to", df_weekly['Date'].max())
    
    # Pivot dataframe so that index is date and columns are clades
    try:
        df_weekly = df_weekly.pivot(index='Date', columns='Variant', values='Freq')
    except Exception as e:
        print("Pivot Error:", e)
        return

    df_weekly = df_weekly.reset_index()
    
    # Debug print to check DataFrame structure
    print("After pivot and reset_index:\n", df_weekly.head())
    
    # Add random collection date for the sake of the example
    df_weekly['Collection date'] = np.random.randint(1, 6, df_weekly.shape[0])
    
    # Split date column to retrieve year and month
    df_weekly['YearMonth'] = df_weekly['Date'].dt.to_period('M').astype(str)
    
    # Rearrange columns
    cols = df_weekly.columns.tolist()
    cols = cols[0:1] + cols[-1:] + cols[1:-1]
    df_weekly = df_weekly[cols]
    
    # Debug print to check DataFrame structure
    print("After rearranging columns:\n", df_weekly.head())
    
    # Specify time frame to analyze
    df_sum = time_frame(df_weekly, start, end)
    
    # Calculate log comparisons at each week
    df_ = clade_comparisons(df_sum)
    df_['days'] -= df_['days'].iloc[0]
    
    # Initialize variables to track the best fit
    best_fit = {
        'slope': None,
        'intercept': None,
        'r_squared': -1,  # Initialize to -1 to ensure any R squared is higher
        'variant': None,
        'start_idx': None,
        'end_idx': None
    }
    
    # Iterate through possible start and end indices within the given range
    for start_idx in range(len(df_)):
        for end_idx in range(start_idx + 1, len(df_)):
            df_subset = df_.iloc[start_idx:end_idx+1]
            if len(df_subset) < 2:
                continue
            
            # Iterate through specified variants to find the best fit
            pango_compare = [variant]
            for i in pango_compare:
                if i != 'Omicron':
                    mask = ~np.isnan(df_subset['days']) & ~np.isnan(df_subset['Omicron vs {0}'.format(i)])
                    if mask.sum() < 2:
                        continue
                    slope, intercept, r_value, p_value, std_err = stats.linregress(df_subset['days'][mask], df_subset['Omicron vs {0}'.format(i)][mask])
                    
                    if r_value ** 2 > best_fit['r_squared'] and p_value < 0.05:
                        best_fit = {
                            'slope': slope,
                            'intercept': intercept,
                            'r_squared': r_value ** 2,
                            'variant': i,
                            'start_idx': start_idx,
                            'end_idx': end_idx
                        }
    
    if best_fit['variant'] is not None:
        # Plot the best fit line for the optimal range
        df_best_fit = df_.iloc[best_fit['start_idx']:best_fit['end_idx']+1]
        mask = ~np.isnan(df_best_fit['days']) & ~np.isnan(df_best_fit['Omicron vs {0}'.format(best_fit['variant'])])
        sns.set_style("white")
        sns.set(rc={'figure.figsize': (3, 3)})
        sns.regplot(
            x=df_best_fit['days'], y=df_best_fit['Omicron vs {0}'.format(best_fit['variant'])],
            marker='o', scatter_kws={'s': 50},
            label='Omicron vs {0}: y={1:.1e}x+{2:.1f}'.format(best_fit['variant'], best_fit['slope'], best_fit['intercept']),
            color='red'
        )
    
        plt.legend(loc='upper left', prop={'size': 6}, facecolor='white', frameon=False)
        plt.ylabel(r'$\log_{10}\left(\frac{\mathrm{Variant\ 1}}{\mathrm{Variant\ 2}}\right)$', fontsize=7.5, labelpad=1)
        plt.title(f'Omicron vs {best_fit["variant"]}\nOptimal Time Window: ({df_best_fit["days"].iloc[0]}) - ({df_best_fit["days"].iloc[-1]})\nLocation: {country}', fontsize=7.5)
        plt.xlabel('Days', fontsize=7.5)
        plt.gca().set_facecolor('white')
        plt.axhline(y=plt.ylim()[0], color=(0, 0, 0.1))
        plt.axvline(x=plt.xlim()[0], color=(0, 0, 0.1))
        
        # Save the figures to `omicron_vs_delta_regression_plots` folder
        import os
        
#         # Specify the directory path
#         directory = "/scr/mjpantho/Pantho_differential_fitness/omicron_vs_delta_regression_plots/"
        
#         # Create the directory if it doesn't exist
#         if not os.path.exists(directory):
#             os.makedirs(directory)
        
#         # Save the figure to the specified directory with DPI set to 300
#         filename = os.path.join(directory, country + "_regression_Omicron_vs_" + best_fit['variant'] + ".png")
#         plt.savefig(filename, dpi=300, bbox_inches="tight")
    else:
        print("No valid fit found within the specified date range and criteria.")

# Helper functions used within scatterplot
def time_frame(df, start_date, end_date):  # updated
    time_window = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m').unique().tolist()
    print(time_window)
    dataframe = df.loc[df['YearMonth'].isin(time_window)]
    dataframe.iloc[:,2:] = dataframe.iloc[:,2:].cumsum()
    for i in dataframe.columns.tolist():
        if dataframe[i].isnull().all():
            dataframe = dataframe.drop(i, axis=1)
    return dataframe

def time_frame_no_cumsum(df, start_date, end_date):  # updated
    time_window = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m').unique().tolist()
    print(time_window)
    dataframe = df.loc[df['YearMonth'].isin(time_window)]
    return dataframe

def transform_ratio(clade_1, clade_2):
    return np.log10(clade_1/clade_2)

def clade_comparisons(df):
    clade_list = [i for i in clade_input if i in df.columns.tolist()]

    df['date_ordinal'] = pd.to_datetime(df['Date']).apply(lambda date: date.toordinal())
    df['days'] = df['date_ordinal'] - 737425
    # Calculate log ratio for each clade comparison
    for i in clade_list:
        for j in clade_list:
            if i != j:
                df['{a} vs {b}'.format(a=i,b=j)] = df.apply(lambda row : 
                                                                        transform_ratio(row['{a}'.format(a=i)],
                                                                                        row['{b}'.format(b=j)]),
                                                                                        axis = 1)
    clade_list.append('date_ordinal')
    df = df.drop(clade_list, axis=1)
    return df


# scatterplot('USA', df_weekly_sum, start='2020-01-01', end='2020-05-01', variant='Delta')


In [39]:
scatterplot('USA', df_weekly_sum, start = '2020-03-01', end = '2022-05-01', variant = 'Delta')  #you can pass a list of variants to compare

Unique Variants: ['Alpha' 'Beta' 'Delta' 'Gamma' 'Omicron']


ValueError: offset must be a timedelta strictly between -timedelta(hours=24) and timedelta(hours=24).